In [1]:
from datetime import datetime
import os
import re
import glob
import requests 
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np
import folium

In [52]:
link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [53]:
thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [54]:
head_rows = []
body_rows = []
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [55]:
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         
df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"2,774,864","+5,864","156,223",+808,"1,250,855","+5,502","1,367,786","19,422",,,,,,North America,\n,,
1,,\nSouth America\n,"1,817,322","+1,105","72,629",+47,"1,022,055","+15,307","722,638","12,536",,,,,,South America,\n,,
2,,\nEurope\n,"2,327,734",+887,"187,955",+26,"1,266,443",+344,"873,336","6,198",,,,,,Europe,\n,,
3,,\nAsia\n,"1,946,883","+6,209","49,191",+143,"1,240,026","+3,697","657,666","19,249",,,,,,Asia,\n,,
4,,\nAfrica\n,"318,321",,"8,371",,"153,242",,"156,708",502,,,,,,Africa,\n,,


In [56]:
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)
df_bs = df_bs.drop('#', axis=1)
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"2,388,225",+72,"122,611",+1,"1,003,062",+133,"1,262,552","16,510","7,216",370,"29,027,198","87,706","330,959,930",North America
1,Brazil,"1,111,348",,"51,407",,"594,104","+14,878","465,837","8,318","5,229",242,"2,510,717","11,814","212,525,202",South America
2,Russia,"592,280",,"8,206",,"344,416",,"239,658","2,300","4,059",56,"17,289,691","118,477","145,933,256",Europe
3,India,"441,070",+620,"14,019",+4,"248,286",+149,"178,765","8,944",320,10,"7,137,716","5,173","1,379,678,183",Asia
4,UK,"305,289",,"42,647",,N/A,N/A,N/A,330,"4,498",628,"8,029,757","118,296","67,878,273",Europe


In [57]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"330,959,930","2,388,225",+72,"122,611",+1,"1,003,062",+133,"1,262,552","16,510","7,216",370,"29,027,198","87,706"
1,Brazil,South America,"212,525,202","1,111,348",,"51,407",,"594,104","+14,878","465,837","8,318","5,229",242,"2,510,717","11,814"
2,Russia,Europe,"145,933,256","592,280",,"8,206",,"344,416",,"239,658","2,300","4,059",56,"17,289,691","118,477"
3,India,Asia,"1,379,678,183","441,070",+620,"14,019",+4,"248,286",+149,"178,765","8,944",320,10,"7,137,716","5,173"
4,UK,Europe,"67,878,273","305,289",,"42,647",,N/A,N/A,N/A,330,"4,498",628,"8,029,757","118,296"


In [58]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [59]:
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['CAR', 'French Guiana', 'Mayotte', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Cayman Islands', 'Faeroe Islands', 'Gibraltar', 'Guadeloupe',
       'Brunei ', 'Aruba', 'Sint Maarten', 'French Polynesia', 'Macao',
       'Saint Martin', 'St. Vincent Grenadines', 'Curaçao',
       'New Caledonia', 'Turks and Caicos', 'Falkland Islands',
       'Montserrat', 'MS Zaandam'], dtype=object)

In [60]:
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)
df_bs = df_bs.replace('', np.nan)
df_bs = df_bs.fillna("Not Available")
df_bs = df_bs.astype(str)
df_bs.drop(columns=['NewCases','NewDeaths','NewRecovered','Serious,Critical'],axis=1,inplace=True)
df_bs.sort_values(by=["Country/Region"],inplace=True)
df_bs.drop(df_bs[df_bs["Continent"]=="Not Available"].index,axis=0,inplace=True)
df_bs = df_bs.reset_index(drop=True)

In [61]:
df_bs["Country/Region"][50]

'Djibouti'

In [62]:
df_bs.to_csv('worldometer_data.csv', index=False)

In [63]:
countries = pd.read_html("https://developers.google.com/public-data/docs/canonical/countries_csv")

In [64]:
countries = pd.DataFrame(countries[0])
countries.sort_values("name",inplace=True)
countries = countries.reset_index(drop=True)
countries

,country,latitude,longitude,name
0,AF,33.939110,67.709953,Afghanistan
1,AL,41.153332,20.168331,Albania
2,DZ,28.033886,1.659626,Algeria
3,AS,-14.270972,-170.132217,American Samoa
4,AD,42.546245,1.601554,Andorra
...,...,...,...,...
240,WF,-13.768752,-177.156097,Wallis and Futuna
241,EH,24.215527,-12.885834,Western Sahara
242,YE,15.552727,48.516388,Yemen
243,ZM,-13.133897,27.849332,Zambia


In [191]:
lat, long = [], []
count = 0
for i in countries["name"]:
    for j in df_bs["Country/Region"]:
        if i==j:
            lat.append((i,countries["latitude"][count]))
            long.append((i,countries["longitude"][count]))
    
    count += 1
            
print(count)  
print(len(set(lat)))

245
200


In [192]:
lat.extend([('Brunei',4.5353),('Channel Islands',49.372284),('Curaçao',12.1696),('Eswatini',-26.5225),('Saint Martin',18.0826),('Sint Maarten',18.0425),('Republic of South Sudan',6.8770)])
long.extend([('Brunei',114.7277),('Channel Islands',-2.364351),('Curaçao',-68.9900),('Eswatini',31.4659),('Saint Martin',-63.0501),('Sint Maarten',-63.0548),('Republic of South Sudan',31.3070)])

In [193]:
lat

[('Afghanistan', 33.93911),
 ('Albania', 41.153332),
 ('Algeria', 28.033886),
 ('Andorra', 42.546245),
 ('Angola', -11.202691999999999),
 ('Antigua and Barbuda', 17.060816),
 ('Argentina', -38.416097),
 ('Armenia', 40.069099),
 ('Aruba', 12.52111),
 ('Australia', -25.274398),
 ('Austria', 47.516231),
 ('Azerbaijan', 40.143105),
 ('Bahamas', 25.03428),
 ('Bahrain', 25.930414000000003),
 ('Bangladesh', 23.684994),
 ('Barbados', 13.193887),
 ('Belarus', 53.709807),
 ('Belgium', 50.503887),
 ('Belize', 17.189877),
 ('Benin', 9.30769),
 ('Bermuda', 32.321384),
 ('Bhutan', 27.514162),
 ('Bolivia', -16.290154),
 ('Bosnia and Herzegovina', 43.915886),
 ('Botswana', -22.328474),
 ('Brazil', -14.235004),
 ('Bulgaria', 42.733883),
 ('Burkina Faso', 12.238333),
 ('Burundi', -3.3730559999999996),
 ('Cambodia', 12.565679),
 ('Cameroon', 7.369722),
 ('Canada', 56.130366),
 ('Cape Verde', 16.002082),
 ('Cayman Islands', 19.513469),
 ('Central African Republic', 6.611111),
 ('Chad', 15.454166),
 ('Chil

In [194]:
df_bs.replace(df_bs["Country/Region"][30],"Central African Republic",inplace=True)
df_bs.replace(df_bs["Country/Region"][31],"Cape Verde",inplace=True)
df_bs.replace(df_bs["Country/Region"][41],"Congo [Republic]",inplace=True)
df_bs.replace(df_bs["Country/Region"][47],"Czech Republic",inplace=True)
df_bs.replace(df_bs["Country/Region"][48],"Congo [DRC]",inplace=True)
df_bs.replace(df_bs["Country/Region"][62-1],"Faroe Islands",inplace=True)
df_bs.replace(df_bs["Country/Region"][63-1],"Falkland Islands [Islas Malvinas]",inplace=True)
df_bs.replace(df_bs["Country/Region"][96-1],"Côte d'Ivoire",inplace=True)
df_bs.replace(df_bs["Country/Region"][114-2],"Macau",inplace=True)
df_bs.replace(df_bs["Country/Region"][133-2],"Myanmar [Burma]",inplace=True)
df_bs.replace(df_bs["Country/Region"][142-2],"Macedonia [FYROM]",inplace=True)
df_bs.replace(df_bs["Country/Region"][146-2],"Palestinian Territories",inplace=True)
df_bs.replace(df_bs["Country/Region"][159-2],"South Korea",inplace=True)
df_bs.replace(df_bs["Country/Region"][164-2],"São Tomé and Príncipe",inplace=True)
df_bs.replace(df_bs["Country/Region"][176-2],"Republic of South Sudan",inplace=True)
df_bs.replace(df_bs["Country/Region"][179-2],"Saint Vincent and the Grenadines",inplace=True)
df_bs.replace(df_bs["Country/Region"][192],"Turks and Caicos Islands",inplace=True)
df_bs.replace(df_bs["Country/Region"][193],"United Arab Emirates",inplace=True)
df_bs.replace(df_bs["Country/Region"][194],"United Kingdom",inplace=True)
df_bs.replace(df_bs["Country/Region"][195],"United States",inplace=True)

In [195]:
df_bs

,Country/Region,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region,d
0,Afghanistan,Asia,38901392,29481,618,9260,19603,758,16,66800,1717,EasternMediterranean,True
1,Albania,Europe,2877855,1995,44,1159,792,693,15,20762,7214,Europe,True
2,Algeria,Africa,43828594,11920,852,8559,2509,272,19,Not Available,Not Available,Africa,True
3,Andorra,Europe,77263,855,52,796,7,11066,673,3750,48536,Europe,True
4,Angola,Africa,32830010,186,10,77,99,6,0.3,10000,305,Africa,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Vietnam,Asia,97317898,349,Not Available,328,21,4,Not Available,275000,2826,WesternPacific,True
203,Western Sahara,Africa,596871,10,1,8,1,17,2,Not Available,Not Available,Africa,True
204,Yemen,Asia,29805944,967,257,350,360,32,9,120,4,EasternMediterranean,True
205,Zambia,Africa,18366582,1430,11,1194,225,78,0.6,47968,2612,Africa,True


In [196]:
lat.sort()
long.sort()

In [200]:
long

[('Afghanistan', 67.709953),
 ('Albania', 20.168331),
 ('Algeria', 1.6596259999999998),
 ('Andorra', 1.6015540000000001),
 ('Angola', 17.873887),
 ('Antigua and Barbuda', -61.796428000000006),
 ('Argentina', -63.616671999999994),
 ('Armenia', 45.038189),
 ('Aruba', -69.968338),
 ('Australia', 133.775136),
 ('Austria', 14.550072),
 ('Azerbaijan', 47.576927000000005),
 ('Bahamas', -77.39628),
 ('Bahrain', 50.637772),
 ('Bangladesh', 90.35633100000001),
 ('Barbados', -59.543198),
 ('Belarus', 27.953389),
 ('Belgium', 4.469936),
 ('Belize', -88.49765),
 ('Benin', 2.315834),
 ('Bermuda', -64.75737),
 ('Bhutan', 90.433601),
 ('Bolivia', -63.588653),
 ('Bosnia and Herzegovina', 17.679076000000002),
 ('Botswana', 24.684866),
 ('Brazil', -51.92528),
 ('Brunei', 114.7277),
 ('Bulgaria', 25.48583),
 ('Burkina Faso', -1.561593),
 ('Burundi', 29.918885999999997),
 ('Cambodia', 104.990963),
 ('Cameroon', 12.354722),
 ('Canada', -106.34677099999999),
 ('Cape Verde', -24.013197),
 ('Cayman Islands', -

In [205]:
latitude, longitude = [], []
for i,j in lat:
    latitude.append(j)
for i,j in long:
    longitude.append(j)

In [206]:
len(longitude)

207

In [207]:
df_bs["Latitude"] = latitude
df_bs["Longitude"] = longitude

In [208]:
df_bs

,Country/Region,Continent,Population,TotalCases,TotalDeaths,TotalRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region,d,Latitude,Longitude
0,Afghanistan,Asia,38901392,29481,618,9260,19603,758,16,66800,1717,EasternMediterranean,True,33.939110,67.709953
1,Albania,Europe,2877855,1995,44,1159,792,693,15,20762,7214,Europe,True,41.153332,20.168331
2,Algeria,Africa,43828594,11920,852,8559,2509,272,19,Not Available,Not Available,Africa,True,28.033886,1.659626
3,Andorra,Europe,77263,855,52,796,7,11066,673,3750,48536,Europe,True,42.546245,1.601554
4,Angola,Africa,32830010,186,10,77,99,6,0.3,10000,305,Africa,True,-11.202692,17.873887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Vietnam,Asia,97317898,349,Not Available,328,21,4,Not Available,275000,2826,WesternPacific,True,14.058324,108.277199
203,Western Sahara,Africa,596871,10,1,8,1,17,2,Not Available,Not Available,Africa,True,24.215527,-12.885834
204,Yemen,Asia,29805944,967,257,350,360,32,9,120,4,EasternMediterranean,True,15.552727,48.516388
205,Zambia,Africa,18366582,1430,11,1194,225,78,0.6,47968,2612,Africa,True,-13.133897,27.849332


In [212]:
india = folium.Map(location = [100,0],zoom_start=2)

for country,lat,long in zip(list(df_bs["Country/Region"]),list(df_bs['Latitude']),list(df_bs['Longitude'])):
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    folium.Marker(location = [lat,long],
                  popup=folium.Popup(('<strong><font color= blue>Country : '+country+'</font></striong><br>' ),max_width=200)).add_to(india)
india